In [42]:
import numpy as np
from numpy import ndarray
from numpy.linalg import inv, norm
import matplotlib.pyplot as plt
import math
import time
from typing import Callable, Tuple, Union, List
import pandas as pd

# Question 1

## 1a

In [7]:
def f(x, y):
    return 3*x**2 - y**2

def g(x, y):
    return 3*x*y**2 - x**3 - 1

In [9]:
def fixed_iteration(x0, y0, max_iter=100, tol=1e-8):
    jac = np.array([[1/6, 1/18],
                  [   0, 1/6  ]])
    
    x, y = x0, y0
    for n in range(max_iter):
        F_val = f(x, y)
        G_val = g(x, y)
        
        dx = jac[0, 0]*F_val + jac[0, 1]*G_val
        dy = jac[1, 0]*F_val + jac[1, 1]*G_val
        
        x_new = x - dx
        y_new = y - dy
        
        diff = np.sqrt((x_new - x)**2 + (y_new - y)**2)
        if diff < tol:
            return x_new, y_new, n+1
        
        x, y = x_new, y_new
    
    return x, y, max_iter



In [8]:
x0, y0 = 1.0, 1.0
x_sol, y_sol, iters = fixed_iteration(x0, y0, max_iter=1000, tol=1e-12)
print(f"Converged in {iters} iterations to x = {x_sol}, y = {y_sol}")
print(f"f(x,y) = {f(x_sol, y_sol)}, g(x,y) = {g(x_sol, y_sol)}")

Converged in 41 iterations to x = 0.4999999999999763, y = 0.8660254037837039
f(x,y) = 1.201594379551807e-12, g(x,y) = -1.944444605328499e-12


## 1c

In [14]:

def evalF(X):
    x, y = X
    f_val = 3*x**2 - y**2
    g_val = 3*x*y**2 - x**3 - 1
    return np.array([f_val, g_val])


def evalJ(X):
    x, y = X
    return np.array([
        [6*x,        -2*y],
        [3*y**2 - 3*x**2,  6*x*y]
    ])

In [15]:
def Newton(x0, tol, Nmax):
    for its in range(Nmax):
        J = evalJ(x0)
        Jinv = inv(J)
        F = evalF(x0)

        x1 = x0 - Jinv.dot(F)

        if (norm(x1-x0) < tol):
            xstar = x1
            ier = 0
            return [xstar, ier, its]

        x0 = x1

    xstar = x1
    ier = 1
    return [xstar, ier, its]

In [16]:
x0 = np.array([1.0, 1.0])
tol = 1e-12
Nmax = 50

xstar, ier, its = Newton(x0, tol, Nmax)
print(f"Solution: x = {xstar[0]}, y = {xstar[1]}")
print(f"Converged in {its} iterations, ier = {ier}")
print(f"F(xstar) = {evalF(xstar)}")

Solution: x = 0.5, y = 0.8660254037844386
Converged in 5 iterations, ier = 0
F(xstar) = [ 1.11022302e-16 -2.22044605e-16]


# Question 3

## 3b

In [49]:
def f(x, y, z):
    return x**2 + 4*y**2 + 4*z**2 - 16


def grad_f(x, y, z):
    df_x = 2*x
    df_y = 8*y
    df_z = 8*z
    return np.array([df_x, df_y, df_z])


def normal_iteration_3d(x0, y0, z0, tol=1e-10, max_iter=50):
    x, y, z = x0, y0, z0
    history = []

    f_old = f(x, y, z)
    history.append((0, x, y, z, f_old, None))  

    for n in range(1,max_iter):
        fx, fy, fz = grad_f(x, y, z)
        denom = fx**2 + fy**2 + fz**2

        d = f_old / denom

        x_new = x - d*fx
        y_new = y - d*fy
        z_new = z - d*fz

        f_new = f(x_new, y_new, z_new)

        if abs(f_old) > 0:
            ratio = abs(f_new) / (f_old**2)
        else:
            ratio = None

        history.append((n, x_new, y_new, z_new, f_new, ratio))

        if abs(f_new) < tol:
            return (x_new, y_new, z_new), history, n+1

        x, y, z = x_new, y_new, z_new
        f_old = f_new

    return (x, y, z), history, max_iter


In [50]:
x0, y0, z0 = 1.0, 1.0, 1.0

(solution_x, solution_y, solution_z), hist, iter = normal_iteration_3d(x0, y0, z0, tol=1e-10, max_iter=50)

df = pd.DataFrame(hist, columns=['Iteration', 'x', 'y', 'z', 'f(x,y,z)', 'Ratio'])

display(df)

print(f"\nConverged to x = {solution_x:.15g}, y = {solution_y:.15g}, z = {solution_z:.15g}")
print(f"Final residual f(x, y, z) = {f(solution_x, solution_y, solution_z):.15g}")
print(f"Converged in {iter} iterations, ier = {ier}")


,Iteration,x,y,z,"f(x,y,z)",Ratio
0,0,1.000000,1.000000,1.000000,-7.000000e+00,NaN
1,1,1.106061,1.424242,1.424242,1.451102e+00,0.029614
2,2,1.093926,1.361742,1.361742,3.139797e-02,0.014911
3,3,1.093642,1.360329,1.360329,1.604374e-05,0.016274
4,4,1.093642,1.360328,1.360328,4.199308e-12,0.016314



Converged to x = 1.09364231738823, y = 1.36032838322323, z = 1.36032838322323
Final residual f(x, y, z) = 4.19930756834219e-12
Converged in 5 iterations, ier = 0
